In [1]:
!pip install -q openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s 

In [2]:
import os
import json
import torch
import whisper

# Load Whisper model (choose size depending on your system: tiny, base, small, medium, large)
model = whisper.load_model("medium")  # or "small", "large", etc.

# Define folder and output
folder_path = '/content/drive/MyDrive/ssmt_project/audio/test'   # Folder containing .wav files
output_json = '/content/drive/MyDrive/ssmt_project/punctuation_prediction_model/test_word_boundaries.json'

# Get list of all .wav files in the folder
filenames_list = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
print(f"Found {len(filenames_list)} .wav files in {folder_path}.")

# Prepare results
result = {}
i=1
# Process each .wav file
for filename in filenames_list:
    file_path = os.path.join(folder_path, filename)

    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue

    try:
        # Load and prepare audio
        audio = whisper.load_audio(file_path)
        audio = whisper.pad_or_trim(audio)
        mel = whisper.log_mel_spectrogram(audio).to(model.device)

        # Detect language (optional, can remove if you are sure it's English)
        _, probs = model.detect_language(mel)
        print(f"{i}Detected language for {filename}: {max(probs, key=probs.get)}")
        i=i+1
        # Transcribe with word timestamps
        result_transcription = model.transcribe(file_path, word_timestamps=True, language="en")

        # Extract word timestamps
        word_timestamps = []
        for segment in result_transcription["segments"]:
            for word in segment["words"]:
                word_timestamps.append({
                    "start_time": word["start"],
                    "end_time": word["end"],
                    "word": word["word"]
                })

        # Save the result for this file
        result[filename] = word_timestamps

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Save all results to a JSON file
with open(output_json, 'w') as f:
    json.dump(result, f, indent=4)

print(f"Saved all word boundaries to {output_json}")


100%|█████████████████████████████████████| 1.42G/1.42G [00:24<00:00, 63.4MiB/s]


Found 647 .wav files in /content/drive/MyDrive/ssmt_project/audio/test.
1Detected language for 5111153817746112904.wav: en
2Detected language for 5196260640941390313.wav: en
3Detected language for 14077379425479054842.wav: en
4Detected language for 17541705185473334415.wav: en
5Detected language for 13031516147694203229.wav: en
6Detected language for 1146643745158029931.wav: en
7Detected language for 1116140877447157610.wav: en
8Detected language for 5910253289107526048.wav: en
9Detected language for 2024594662850694945.wav: en
10Detected language for 3111983050474671345.wav: en
11Detected language for 6243143230365031162.wav: en
12Detected language for 14738234113419638776.wav: en
13Detected language for 18359063029191000694.wav: en
14Detected language for 559615000143317446.wav: en
15Detected language for 18080893208509673354.wav: en
16Detected language for 10899527843662463911.wav: en
17Detected language for 1616366247253366320.wav: en
18Detected language for 7587276269543309929.wav

In [3]:
import json

# Path to your JSON file
json_file_path = '/content/drive/MyDrive/ssmt_project/punctuation_prediction_model/test_word_boundaries.json'  # 🔁 Replace with actual filename

# Load the JSON data
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Count the number of entries
entry_count = len(data)

print(f"Number of entries in the JSON file: {entry_count}")


Number of entries in the JSON file: 647
